1. Load the Data Set
2. Define MLP Structure
3. Initialize Model Parameters
4. Forward Propagation
5. Compute Cost
6.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    classification_report

In [4]:
df = pd.read_csv('datasets/BankNote_Authentication.csv')

In [5]:
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1372 non-null   float64
 1   skewness  1372 non-null   float64
 2   curtosis  1372 non-null   float64
 3   entropy   1372 non-null   float64
 4   class     1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [7]:
df.describe()

,variance,skewness,curtosis,entropy,class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [8]:
df.isnull().sum()

variance    0
skewness    0
curtosis    0
entropy     0
class       0
dtype: int64

In [9]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [11]:
print("X type is " + str(type(X)))
print("X shape is " + str(X.shape))
print("y type is " + str(type(y)))
print("y shape is " + str(y.shape))

X type is <class 'pandas.core.frame.DataFrame'>
X shape is (1372, 4)
y type is <class 'pandas.core.series.Series'>
y shape is (1372,)


In [12]:
X = X.to_numpy()

In [13]:
y = y.to_numpy().reshape(-1, 1)

In [14]:
print("X type is " + str(type(X)))
print("X shape is " + str(X.shape))
print("y type is " + str(type(y)))
print("y shape is " + str(y.shape))

X type is <class 'numpy.ndarray'>
X shape is (1372, 4)
y type is <class 'numpy.ndarray'>
y shape is (1372, 1)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
print("X type is " + str(type(X)))
print("X shape is " + str(X.shape))
print("y type is " + str(type(y)))
print("y shape is " + str(y.shape))

X type is <class 'numpy.ndarray'>
X shape is (1372, 4)
y type is <class 'numpy.ndarray'>
y shape is (1372, 1)


## Define Network Structure

In [17]:
def initialize_parameters(n_x, n_h, n_y=1):
    np.random.seed(42)
    w1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    w2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))

    parameters = {
        "w1": w1,
        "b1": b1,
        "w2": w2,
        "b2": b2,
    }

    return parameters

In [18]:
X_train.shape

(1097, 4)

In [19]:
test_parameters = initialize_parameters(X.shape[1], 5, 1)

In [20]:
print("W1 = " + str(test_parameters["w1"]))
print("b1 = " + str(test_parameters["b1"]))
print("W2 = " + str(test_parameters["w2"]))
print("b2 = " + str(test_parameters["b2"]))

W1 = [[ 0.00496714 -0.00138264  0.00647689  0.0152303 ]
 [-0.00234153 -0.00234137  0.01579213  0.00767435]
 [-0.00469474  0.0054256  -0.00463418 -0.0046573 ]
 [ 0.00241962 -0.0191328  -0.01724918 -0.00562288]
 [-0.01012831  0.00314247 -0.00908024 -0.01412304]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
W2 = [[ 0.01465649 -0.00225776  0.00067528 -0.01424748 -0.00544383]]
b2 = [[0.]]


In [21]:
print("W1 = " + str(test_parameters["w1"].shape))
print("b1 = " + str(test_parameters["b1"].shape))
print("W2 = " + str(test_parameters["w2"].shape))
print("b2 = " + str(test_parameters["b2"].shape))

W1 = (5, 4)
b1 = (5, 1)
W2 = (1, 5)
b2 = (1, 1)


## Forward Propogation

In [22]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [23]:
def forward_propagation(X, parameters):
    W1 = parameters["w1"]
    b1 = parameters["b1"]
    W2 = parameters["w2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1, X.T) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    cache = {
        "Z1": Z1,
        "A1": A1,
        "Z2": Z2,
        "A2": A2,
    }

    return A2, cache

In [24]:
test_A2, test_cache = forward_propagation(X_train, test_parameters)

In [25]:
print("A2 test:" + str(test_A2))
print("A2 test shape:" + str(test_A2.shape))

A2 test:[[0.50004348 0.5003625  0.50046776 ... 0.50019115 0.50022992 0.49971893]]
A2 test shape:(1, 1097)


In [26]:
print("Z1 is:" + str(test_cache["Z1"]))
print("A1 is:" + str(test_cache["A1"]))
print("Z2 is:" + str(test_cache["Z2"]))
print("A2 is:" + str(test_cache["A2"]))

Z1 is:[[-0.102913    0.05031256  0.02529391 ...  0.06187844  0.00323155
  -0.05184445]
 [-0.03184479  0.21452315  0.0202957  ...  0.1732279  -0.00842498
  -0.08684872]
 [ 0.09539668 -0.07726144 -0.00443225 ... -0.08418797  0.00827182
   0.04041363]
 [-0.16539789 -0.05743321 -0.09467096 ...  0.0139666  -0.05682372
   0.01772067]
 [ 0.14697393 -0.07822253 -0.03756487 ... -0.09243179 -0.00716118
   0.06171007]]
A1 is:[[-0.10255121  0.05027015  0.02528852 ...  0.06179958  0.00323153
  -0.05179805]
 [-0.03183403  0.21129183  0.02029291 ...  0.17151571 -0.00842478
  -0.08663102]
 [ 0.09510835 -0.07710808 -0.00443222 ... -0.08398964  0.00827163
   0.04039165]
 [-0.16390598 -0.05737015 -0.09438914 ...  0.01396569 -0.05676263
   0.01771882]
 [ 0.14592472 -0.07806337 -0.03754721 ... -0.09216945 -0.00716106
   0.06163186]]
Z2 is:[[ 0.00017392  0.00145001  0.00187104 ...  0.00076458  0.00091968
  -0.00112427]]
A2 is:[[0.50004348 0.5003625  0.50046776 ... 0.50019115 0.50022992 0.49971893]]


## Compute Cost

In [27]:
print("Y shape is " + str(y_train.shape))

Y shape is (1097, 1)


In [28]:
def compute_cost(A2, y):
    m = A2.shape[1]
    cost = - (np.dot(np.log(A2), y) + np.dot(np.log(1 - A2), (1 - y))) / m
    cost = float(np.squeeze(cost))
    return cost

In [29]:
test_cost = compute_cost(test_cache["A2"], y_train)

In [30]:
test_cost

0.6934831763208149

## Backpropagation

In [31]:
print("A1 shape:" + str(test_cache["A1"]))
print("A2 shape:" + str(test_cache["A2"]))
print("W1 shape:" + str(test_parameters["w1"]))
print("W2 shape:" + str(test_parameters["w2"]))
print("x train shape:" + str(X_train.shape))
print("y train shape:" + str(y_train.shape))

A1 shape:[[-0.10255121  0.05027015  0.02528852 ...  0.06179958  0.00323153
  -0.05179805]
 [-0.03183403  0.21129183  0.02029291 ...  0.17151571 -0.00842478
  -0.08663102]
 [ 0.09510835 -0.07710808 -0.00443222 ... -0.08398964  0.00827163
   0.04039165]
 [-0.16390598 -0.05737015 -0.09438914 ...  0.01396569 -0.05676263
   0.01771882]
 [ 0.14592472 -0.07806337 -0.03754721 ... -0.09216945 -0.00716106
   0.06163186]]
A2 shape:[[0.50004348 0.5003625  0.50046776 ... 0.50019115 0.50022992 0.49971893]]
W1 shape:[[ 0.00496714 -0.00138264  0.00647689  0.0152303 ]
 [-0.00234153 -0.00234137  0.01579213  0.00767435]
 [-0.00469474  0.0054256  -0.00463418 -0.0046573 ]
 [ 0.00241962 -0.0191328  -0.01724918 -0.00562288]
 [-0.01012831  0.00314247 -0.00908024 -0.01412304]]
W2 shape:[[ 0.01465649 -0.00225776  0.00067528 -0.01424748 -0.00544383]]
x train shape:(1097, 4)
y train shape:(1097, 1)


In [44]:
def backpropagation(X, y, cache, parameters):
    m = X.shape[1]
    W1 = parameters["w1"]
    W2 = parameters["w2"]
    A1 = cache["A1"]
    A2 = cache["A2"]

    dZ2 = A2.T - y
    dW2 = np.dot(dZ2.T, A1.T) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dZ1 = np.dot(dZ2, W2) * (1 - np.power(A1, 2)).T
    dW1 = np.dot(dZ1.T, X) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    grads = {
        "dW1": dW1,
        "dW2": dW2,
        "db1": db1,
        "db2": db2,
    }

    return grads

In [45]:
test_grads = backpropagation(X_train, y_train, test_cache, test_parameters)

In [46]:
print("dW1 shape: " + str(test_grads["dW1"].shape))
print("dW2 shape: " + str(test_grads["dW2"].shape))
print("db1 shape: " + str(test_grads["db1"].shape))
print("db2 shape: " + str(test_grads["db2"].shape))

dW1 shape: (5, 4)
dW2 shape: (1, 5)
db1 shape: (1, 5)
db2 shape: (1, 1)


In [47]:
def update_parameters(parameters, grads, learning_rate=0.01):
    W1 = parameters["w1"]
    b1 = parameters["b1"]
    W2 = parameters["w2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    dW2 = grads["dW2"]
    db1 = grads["db1"]
    db2 = grads["db2"]

    w1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1.T
    w2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = {
        "w1": w1,
        "b1": b1,
        "w2": w2,
        "b2": b2,
    }

    return parameters

In [48]:
def nn_model(X, Y, n_x, n_h, n_y, n_steps=1000, print_cost=True):
    parameters = initialize_parameters(n_x, n_h, n_y)

    for i in range(0, n_steps):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backpropagation(X, Y, cache, parameters)
        parameters = update_parameters(parameters, grads)

        if print_cost and i % 1000 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return parameters

In [49]:
def predict(parameters, x):
    A2, cache = forward_propagation(x, parameters)
    predicts = A2 > 0.5
    return predicts

In [50]:
X_train.shape

(1097, 4)

In [51]:
parameters = nn_model(X_train, y_train, X_train.shape[1], n_h=6, n_y=1, n_steps=1000)

Cost after iteration 0: 0.693000


In [52]:
predicts = predict(parameters, X_test)

In [53]:
y_true = y_test.flatten()
y_pred = predicts.flatten()
acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')
conf_matrix = confusion_matrix(y_true, y_pred)

In [54]:
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000

Confusion Matrix:
[[153   0]
 [  0 122]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       153
           1       1.00      1.00      1.00       122

    accuracy                           1.00       275
   macro avg       1.00      1.00      1.00       275
weighted avg       1.00      1.00      1.00       275



In [55]:
parameters_n_h = [i for i in range(3, 11)]
parameters_n_steps = [i for i in range(100, 1100, 100)]
for n_h in parameters_n_h:
    for n_step in parameters_n_steps:
        parameters = nn_model(X_train, y_train, X_train.shape[1], n_h=n_h, n_y=1, n_steps=n_step, print_cost=False)
        predicts = predict(parameters, X_test)
        acc = accuracy_score(y_test.flatten(), predicts.flatten())
        print(f"n_h: {n_h}, n_step: {n_step}, acc: {acc}")

n_h: 3, n_step: 100, acc: 1.0
n_h: 3, n_step: 200, acc: 1.0
n_h: 3, n_step: 300, acc: 1.0
n_h: 3, n_step: 400, acc: 1.0
n_h: 3, n_step: 500, acc: 1.0
n_h: 3, n_step: 600, acc: 1.0
n_h: 3, n_step: 700, acc: 1.0
n_h: 3, n_step: 800, acc: 1.0
n_h: 3, n_step: 900, acc: 1.0
n_h: 3, n_step: 1000, acc: 1.0
n_h: 4, n_step: 100, acc: 1.0
n_h: 4, n_step: 200, acc: 1.0
n_h: 4, n_step: 300, acc: 1.0
n_h: 4, n_step: 400, acc: 1.0
n_h: 4, n_step: 500, acc: 1.0
n_h: 4, n_step: 600, acc: 1.0
n_h: 4, n_step: 700, acc: 1.0
n_h: 4, n_step: 800, acc: 1.0
n_h: 4, n_step: 900, acc: 1.0
n_h: 4, n_step: 1000, acc: 1.0
n_h: 5, n_step: 100, acc: 1.0
n_h: 5, n_step: 200, acc: 1.0
n_h: 5, n_step: 300, acc: 1.0
n_h: 5, n_step: 400, acc: 1.0
n_h: 5, n_step: 500, acc: 1.0
n_h: 5, n_step: 600, acc: 1.0
n_h: 5, n_step: 700, acc: 1.0
n_h: 5, n_step: 800, acc: 1.0
n_h: 5, n_step: 900, acc: 1.0
n_h: 5, n_step: 1000, acc: 1.0
n_h: 6, n_step: 100, acc: 1.0
n_h: 6, n_step: 200, acc: 1.0
n_h: 6, n_step: 300, acc: 1.0
n_h: 6,